In [43]:
import os
import json
vanilla, context, simulation = "/data/jiarui_liu/social_reasoning_rl/social-world-model/data/hitom_results/vanilla_o1-2024-12-17_Hi-ToM_data.json", "/data/jiarui_liu/social_reasoning_rl/social-world-model/data/hitom_results/socialized_context_o1-2024-12-17_processed_hitom_data_debug1.csv", "/data/jiarui_liu/social_reasoning_rl/social-world-model/data/hitom_results/simulation_o1-2024-12-17_processed_hitom_data_debug1.csv"

In [44]:
import pandas as pd
examples = [i for i in range(300)] + [i for i in range(600, 900)]

data, context_data, simulation_data = [], [], []
for i in examples:
    with open(os.path.join(vanilla, f"{i}.json"), "r") as f:
        data.append(json.load(f))
    with open(os.path.join(context, f"{i}.json"), "r") as f:
        context_data.append(json.load(f))
    with open(os.path.join(simulation, f"{i}.json"), "r") as f:
        simulation_data.append(json.load(f))
    
df1_prefixed = pd.DataFrame(data, index=examples).add_prefix('vanilla_')
df2_prefixed = pd.DataFrame(context_data, index=examples).add_prefix('context_')
df3_prefixed = pd.DataFrame(simulation_data, index=examples).add_prefix('simulation_')
df = pd.concat([df1_prefixed, df2_prefixed, df3_prefixed], axis=1)

# Check HiToM Data

In [39]:
import pandas as pd
df = pd.read_csv("/data/jiarui_liu/social_reasoning_rl/social-world-model/data/hitom_data/processed_hitom_data.csv")
df

,prompting_type,deception,story_length,question_order,index,story,question,choices,correct_answer,set_id
0,CoTP,False,1,0,0,"1 Carter, Isabella, Chloe, Jack and Lily enter...",Where is the tomato really?,"A. red_suitcase, B. blue_container, C. red_cra...",red_bottle,0
1,CoTP,False,1,0,1,"1 Isabella, Nathan, Lily, Emma and Elizabeth e...",Where is the tangerine really?,"A. red_treasure_chest, B. blue_envelope, C. bl...",blue_basket,1
2,CoTP,False,1,0,2,"1 Nathan, Chloe, Mila, Amelia and Benjamin ent...",Where is the grapefruit really?,"A. red_bathtub, B. blue_envelope, C. red_box, ...",red_bottle,2
3,CoTP,False,1,0,3,"1 Mia, Benjamin, Elizabeth, Oliver and Liam en...",Where is the green_pepper really?,"A. blue_bottle, B. blue_pantry, C. green_treas...",green_container,3
4,CoTP,False,1,0,4,"1 Jacob, Carter, Isla, Ella and Gracie entered...",Where is the turnip really?,"A. red_bottle, B. blue_bottle, C. red_cupboard...",red_bottle,4
...,...,...,...,...,...,...,...,...,...,...
595,CoTP,True,3,4,895,"1 Alexander, Jackson, Amelia, Nathan and James...",Where does Jackson think Alexander thinks Nath...,"A. blue_box, B. blue_suitcase, C. blue_basket,...",green_drawer,115
596,CoTP,True,3,4,896,"1 Amelia, Gracie, Ella, Lily and Emma entered ...",Where does Amelia think Gracie thinks Ella thi...,"A. blue_suitcase, B. blue_bottle, C. green_bat...",green_pantry,116
597,CoTP,True,3,4,897,"1 Benjamin, Mia, Logan, William and Sophia ent...",Where does Logan think William thinks Sophia t...,"A. red_bathtub, B. red_container, C. blue_box,...",blue_bottle,117
598,CoTP,True,3,4,898,"1 Gracie, Chloe, Liam, Carter and Benjamin ent...",Where does Chloe think Benjamin thinks Liam th...,"A. blue_bottle, B. blue_container, C. blue_cup...",red_bottle,118


# Check Socialized Contexts

In [49]:
import re

def extract_integer(text):
    # 匹配一个或多个连续的数字
    match = re.search(r'\d+', text)
    if match:
        return int(match.group())
    return None

# 测试
print(extract_integer("T12"))          # 输出: 12
print(extract_integer("34 (final)"))   # 输出: 34
print(extract_integer("abc123def"))    # 输出: 123
print(extract_integer("no numbers"))   # 输出: None

12
34
123
None


In [48]:
import os
import json

filepath = "/data/jiarui_liu/social_reasoning_rl/social-world-model/data/hitom_results/socialized_context_o1-2024-12-17_processed_hitom_data.csv"
for file in os.listdir(filepath):
    with open(os.path.join(filepath, file), "r") as f:
        data = json.load(f)
    context =data['socialized_context']['socialized_context']
    for item in context:
        try:
            item['timestep'] = int(item['timestep'])
        except:
            print(file)

754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
754.json
831.json
875.json
200.json
200.json
200.json
200.json
200.json
200.json
200.json
200.json
200.json
200.json
200.json
200.json
200.json
200.json
200.json
200.json
201.json
240.json
240.json
240.json
240.json
240.json
240.json
240.json
240.json
240.json
240.json
240.json
240.json
240.json
240.json
240.json
240.json
851.json
221.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
714.json
845.json
845.json
845.json
845.json
845.json
845.json
845.json
845.json
845.json
845.json
845.json
845.json
845.json
845.json
845.json
845.json
8

# Check Question Formats

In [20]:
df['vanilla_question'].values

array(['Where is the tomato really?', 'Where is the tangerine really?',
       'Where is the grapefruit really?',
       'Where is the green_pepper really?', 'Where is the turnip really?',
       'Where is the lettuce really?', 'Where is the pear really?',
       'Where is the celery really?', 'Where is the melon really?',
       'Where is the pear really?', 'Where is the asparagus really?',
       'Where is the apple really?', 'Where is the celery really?',
       'Where is the cabbage really?', 'Where is the beans really?',
       'Where is the beans really?', 'Where is the cherry really?',
       'Where is the melon really?', 'Where is the peach really?',
       'Where is the green_pepper really?',
       'Where does Chloe really think the tomato is?',
       'Where does Elizabeth really think the tangerine is?',
       'Where does Benjamin really think the grapefruit is?',
       'Where does Elizabeth really think the green_pepper is?',
       'Where does Isla really think the turn

# Error Analysis: Vanilla vs. Context

In [34]:
df[['vanilla_is_correct', 'context_is_correct']].value_counts()

vanilla_is_correct  context_is_correct
True                True                  477
False               False                  48
True                False                  43
False               True                   32
Name: count, dtype: int64

In [35]:
tmp = df.loc[(df['vanilla_is_correct'] == False) & (df['context_is_correct'] == False)]

In [36]:
tmp = df.loc[df['context_is_correct'] == False]
tmp.index

Index([ 80, 132, 146, 156, 160, 162, 164, 173, 177, 180, 185, 192, 195, 197,
       243, 251, 256, 263, 281, 285, 291, 641, 646, 647, 654, 656, 657, 659,
       660, 662, 664, 666, 674, 676, 679, 690, 691, 692, 697, 699, 737, 739,
       750, 752, 758, 759, 769, 770, 777, 778, 779, 781, 784, 790, 794, 796,
       799, 827, 832, 834, 841, 842, 843, 844, 846, 847, 853, 858, 860, 861,
       863, 864, 867, 869, 871, 872, 873, 875, 877, 878, 881, 883, 884, 887,
       889, 891, 892, 893, 895, 897, 898],
      dtype='int64')

In [33]:
to_csv("hitom_vanilla_correct_context_incorrect.csv")

In [36]:
with open("hitom_vanilla_correct_context_incorrect.json", "w") as f:
    json.dump(tmp.to_dict(orient='records'), f, indent=2, ensure_ascii=False)

In [8]:
tmp.columns

Index(['vanilla_story', 'vanilla_question', 'vanilla_reasoning',
       'vanilla_answer', 'vanilla_correct_answer', 'vanilla_choices',
       'vanilla_is_correct', 'context_set_id', 'context_index',
       'context_deception', 'context_story_length', 'context_question_order',
       'context_story', 'context_question', 'context_reasoning',
       'context_answer', 'context_correct_answer',
       'context_socialized_context', 'context_extra_info', 'context_choices',
       'context_is_correct'],
      dtype='object')

In [9]:
id = 2

In [10]:
print(tmp.iloc[id]['context_story'])

1 Mila, Benjamin, Ethan, Isabella and Lily entered the porch.
2 The apple is in the blue_suitcase.
3 Mila made no movements and stayed in the porch for 1 minute.
4 Mila exited the porch.
5 Benjamin moved the apple to the green_bottle.
6 Benjamin exited the porch.
7 Ethan moved the apple to the blue_pantry.
8 Ethan exited the porch.
9 Isabella made no movements and stayed in the porch for 1 minute.
10 Isabella exited the porch.
11 Lily moved the apple to the blue_suitcase.
12 Lily exited the porch.
13 Mila, Benjamin, Ethan, Isabella and Lily entered the waiting_room.
14 Ethan, Lily and Benjamin entered the garden.
15 The beans is in the red_bathtub.
16 Ethan made no movements and stayed in the garden for 1 minute.
17 Ethan exited the garden.
18 Lily made no movements and stayed in the garden for 1 minute.
19 Lily exited the garden.
20 Benjamin made no movements and stayed in the garden for 1 minute.
21 Benjamin exited the garden.
22 Ethan, Lily and Benjamin entered the waiting_room.




In [11]:
tmp.iloc[id]['context_socialized_context']

{'agents_names': ['Mila', 'Benjamin', 'Ethan', 'Isabella', 'Lily'],
 'socialized_context': [{'timestep': '1',
   'state': 'none',
   'observations': {'Mila': 'none',
    'Benjamin': 'none',
    'Ethan': 'none',
    'Isabella': 'none',
    'Lily': 'none'},
   'actions': {'Mila': 'Mila entered the porch',
    'Benjamin': 'Benjamin entered the porch',
    'Ethan': 'Ethan entered the porch',
    'Isabella': 'Isabella entered the porch',
    'Lily': 'Lily entered the porch'}},
  {'timestep': '2',
   'state': 'Mila, Benjamin, Ethan, Isabella, and Lily are all in the porch. The apple is in the blue_suitcase.',
   'observations': {'Mila': '<same_as_state />',
    'Benjamin': '<same_as_state />',
    'Ethan': '<same_as_state />',
    'Isabella': '<same_as_state />',
    'Lily': '<same_as_state />'},
   'actions': {'Mila': 'none',
    'Benjamin': 'none',
    'Ethan': 'none',
    'Isabella': 'none',
    'Lily': 'none'}},
  {'timestep': '3',
   'state': 'Everyone remains in the porch with the appl

In [12]:
tmp.iloc[id]['context_question']

'Where does Mila think Lily thinks Benjamin thinks the apple is?'

In [13]:
print(tmp.iloc[id]['vanilla_reasoning'])

1) Initially, all five (Mila, Benjamin, Ethan, Isabella, Lily) see the apple in the blue_suitcase.  
2) Mila exits first (step 4), so she does not witness any later moves and believes the apple remains in the blue_suitcase.  
3) Benjamin leaves next (step 6), having moved the apple to the green_bottle (step 5). After he leaves, he does not see further moves, so he believes the apple is in the green_bottle.  
4) Ethan then moves the apple to the blue_pantry (step 7) before leaving (step 8).  
5) Lily eventually moves it again to the blue_suitcase (step 11) before leaving last (step 12).  
6) Lily knows Benjamin left right after placing the apple in the green_bottle, so from Lily’s point of view, Benjamin still thinks it is in the green_bottle.  
7) Mila, knowing Lily stayed long after Benjamin, will figure Lily saw Benjamin’s move to the green_bottle and knows Benjamin still thinks the apple is there.  
8) Therefore, “Where does Mila think Lily thinks Benjamin thinks the apple is?” = gr

In [14]:
print(tmp.iloc[id]['context_reasoning'])

Step-by-step reasoning:

1) Actual moves in the porch:  
   - After Mila leaves (she exited first), Benjamin moves the apple from the blue_suitcase → green_bottle, then exits.  
   - Next, Ethan moves it from green_bottle → blue_pantry, then exits.  
   - Lastly, Lily moves it from blue_pantry → blue_suitcase, then exits.

2) Benjamin’s own belief:  
   Since Benjamin left immediately after putting the apple in the green_bottle (and never saw Ethan or Lily’s later moves), Benjamin believes it is still in the green_bottle.

3) Lily’s belief about Benjamin’s belief:  
   Lily was present to see that Benjamin left right after placing it in green_bottle. Hence Lily thinks that Benjamin still believes it is in the green_bottle.

4) Mila’s belief about Lily’s belief about Benjamin:  
   Mila left before any moves, but she knows Benjamin exited after her and Lily exited last. Because Lily remained in the porch longer than Benjamin, Mila can deduce that Lily would realize Benjamin last saw the

In [15]:
print(tmp.iloc[id]['context_choices'])
print(tmp.iloc[id]['context_answer'], tmp.iloc[id]['context_correct_answer'])

A. red_bathtub, B. red_envelope, C. red_container, D. green_treasure_chest, E. red_drawer, F. red_cupboard, G. blue_suitcase, H. blue_pantry, I. green_envelope, J. green_bottle, K. green_container, L. red_treasure_chest, M. blue_basket, N. blue_drawer, O. red_crate
J blue_suitcase


In [16]:
tmp['context_question_order'].value_counts()

context_question_order
3    19
2    15
4    10
1     4
Name: count, dtype: int64

In [17]:
tmp['context_deception'].value_counts()

context_deception
True     43
False     5
Name: count, dtype: int64

In [18]:
tmp['context_story_length'].value_counts()

context_story_length
3    24
1    13
2    11
Name: count, dtype: int64

# Error Analysis: Vanilla vs. Simulation

In [45]:
df[['vanilla_is_correct', 'simulation_is_correct']].value_counts()

vanilla_is_correct  simulation_is_correct
True                True                     434
                    False                     86
False               True                      58
                    False                     22
Name: count, dtype: int64

In [46]:
tmp = df.loc[(df['vanilla_is_correct'] == True) & (df['simulation_is_correct'] == False)]

In [47]:
tmp.index

Index([ 20,  31,  52,  61,  77,  87,  97, 121, 122, 125, 126, 129, 134, 135,
       139, 147, 151, 155, 156, 171, 172, 175, 177, 178, 184, 185, 191, 195,
       198, 220, 224, 225, 230, 242, 243, 262, 265, 276, 280, 283, 285, 296,
       298, 611, 620, 624, 625, 626, 631, 633, 638, 639, 654, 670, 678, 698,
       724, 726, 729, 730, 732, 734, 736, 739, 749, 754, 759, 765, 774, 792,
       794, 823, 824, 826, 829, 832, 833, 849, 852, 854, 857, 859, 863, 876,
       883, 894],
      dtype='int64')

In [10]:
tmp['simulation_question_order'].value_counts()

simulation_question_order
1    36
4    17
2    16
3    16
0     1
Name: count, dtype: int64

In [11]:
tmp['simulation_deception'].value_counts()

simulation_deception
False    43
True     43
Name: count, dtype: int64

In [12]:
tmp['simulation_story_length'].value_counts()

simulation_story_length
2    37
3    29
1    20
Name: count, dtype: int64

In [33]:
id = 2

In [34]:
tmp.iloc[id]

vanilla_story                    The following story happens in chronological o...
vanilla_question                 Where does Oliver think Logan thinks the celer...
vanilla_reasoning                1) Initially, everyone sees the celery in the ...
vanilla_answer                                                                   L
vanilla_correct_answer                                         blue_treasure_chest
vanilla_choices                  A. blue_suitcase, B. blue_box, C. red_bathtub,...
vanilla_is_correct                                                            True
context_set_id                                                                  12
context_index                                                                   52
context_deception                                                            False
context_story_length                                                             1
context_question_order                                                           2
cont

In [35]:
print(tmp.iloc[id]['simulation_story'])

1 Amelia, Logan, Aria, Oliver and Chloe entered the cellar.
2 The celery is in the red_container.
3 Amelia made no movements and stayed in the cellar for 1 minute.
4 Amelia exited the cellar.
5 Logan moved the celery to the blue_treasure_chest.
6 Logan exited the cellar.
7 Aria made no movements and stayed in the cellar for 1 minute.
8 Aria exited the cellar.
9 Oliver made no movements and stayed in the cellar for 1 minute.
10 Oliver exited the cellar.
11 Chloe moved the celery to the red_container.
12 Chloe exited the cellar.
13 Amelia, Logan, Aria, Oliver and Chloe entered the waiting_room.




In [36]:
print(tmp.iloc[id]['simulation_question'])

Where does Oliver think Logan thinks the celery is?


In [37]:
tmp.columns

Index(['vanilla_story', 'vanilla_question', 'vanilla_reasoning',
       'vanilla_answer', 'vanilla_correct_answer', 'vanilla_choices',
       'vanilla_is_correct', 'context_set_id', 'context_index',
       'context_deception', 'context_story_length', 'context_question_order',
       'context_story', 'context_question', 'context_reasoning',
       'context_answer', 'context_correct_answer',
       'context_socialized_context', 'context_extra_info', 'context_choices',
       'context_is_correct', 'simulation_set_id', 'simulation_index',
       'simulation_deception', 'simulation_story_length',
       'simulation_question_order', 'simulation_story', 'simulation_question',
       'simulation_reasoning', 'simulation_answer',
       'simulation_correct_answer', 'simulation_choices',
       'simulation_is_correct', 'simulation_socialized_context',
       'simulation_extra_info'],
      dtype='object')

In [38]:
print(
    tmp.iloc[id]['simulation_choices'])
print(
tmp.iloc[id]['simulation_correct_answer'],
tmp.iloc[id]['vanilla_answer'],
tmp.iloc[id]['context_answer'],
tmp.iloc[id]['simulation_answer'],
)

A. blue_suitcase, B. blue_box, C. red_bathtub, D. red_bottle, E. green_pantry, F. red_crate, G. green_bucket, H. green_treasure_chest, I. blue_container, J. red_envelope, K. red_container, L. blue_treasure_chest, M. green_drawer, N. blue_envelope, O. blue_bathtub
blue_treasure_chest L L K


In [39]:
print(tmp.iloc[id]['context_reasoning'])

Step-by-step reasoning:

1) Initially everyone (including Oliver and Logan) sees the celery in the red_container.  
2) Amelia leaves first. Immediately after, Logan moves the celery to the blue_treasure_chest (Aria, Oliver, and Chloe witness this).  
3) Logan exits next. At the time Logan leaves, he believes the celery remains in the blue_treasure_chest.  
4) Aria then exits (without moving the celery), followed by Oliver who also sees the celery still in the blue_treasure_chest. Oliver leaves next, unaware of any further change.  
5) Only Chloe remains and moves the celery back to the red_container after Oliver is gone. Therefore Oliver does not witness this final move.  

Given that Oliver knows Logan left the cellar while the celery was in the blue_treasure_chest and that no public or private communications occur to change Logan’s belief, Oliver concludes that Logan still thinks the celery is in the blue_treasure_chest.


In [32]:
print(tmp.iloc[id]['simulation_reasoning'])


1. Initially, the apple is in the red_box.  
2. Carter moves it to the blue_basket while Amelia is already outside; hence Liam sees that it is in the blue_basket.  
3. Liam leaves the kitchen next, and only Gracie and Jackson remain for a time.  
4. After Liam is gone, the story states that the apple ends up in the red_box again before Jackson exits.  
5. Since Jackson exited the kitchen after Liam did, and Liam is more inclined to trust someone who left after him, Liam concludes the apple must truly have been moved back into the red_box while he was no longer there.  
6. Therefore, Liam's best conclusion is that the apple is in the red_box.  



In [22]:
tmp.iloc[id]['context_socialized_context']

{'agents_names': ['Carter', 'Isabella', 'Chloe', 'Jack', 'Lily'],
 'socialized_context': [{'timestep': '1',
   'state': 'none',
   'observations': {'Carter': 'none',
    'Isabella': 'none',
    'Chloe': 'none',
    'Jack': 'none',
    'Lily': 'none'},
   'actions': {'Carter': 'Carter entered the sunroom',
    'Isabella': 'Isabella entered the sunroom',
    'Chloe': 'Chloe entered the sunroom',
    'Jack': 'Jack entered the sunroom',
    'Lily': 'Lily entered the sunroom'}},
  {'timestep': '2',
   'state': 'Carter, Isabella, Chloe, Jack, Lily are in the sunroom. The tomato is in the green_treasure_chest in the sunroom.',
   'observations': {'Carter': '<same_as_state />',
    'Isabella': '<same_as_state />',
    'Chloe': '<same_as_state />',
    'Jack': '<same_as_state />',
    'Lily': '<same_as_state />'},
   'actions': {'Carter': 'none',
    'Isabella': 'none',
    'Chloe': 'none',
    'Jack': 'none',
    'Lily': 'none'}},
  {'timestep': '3',
   'state': 'All five are in the sunroom wi

In [23]:
tmp.iloc[id]['simulation_socialized_context']

{'agents_names': ['Carter', 'Isabella', 'Chloe', 'Jack', 'Lily'],
 'socialized_context': [{'timestep': '2',
   'state': 'Carter, Isabella, Chloe, Jack, Lily are in the sunroom. The tomato is in the green_treasure_chest in the sunroom.',
   'observations': {'Carter': '<same_as_state />',
    'Isabella': '<same_as_state />',
    'Chloe': '<same_as_state />',
    'Jack': '<same_as_state />',
    'Lily': '<same_as_state />'},
   'actions': {'Carter': 'none',
    'Isabella': 'none',
    'Chloe': 'none',
    'Jack': 'none',
    'Lily': 'none'}},
  {'timestep': '3',
   'state': 'All five are in the sunroom with the tomato in the green_treasure_chest.',
   'observations': {'Carter': '<same_as_state />',
    'Isabella': '<same_as_state />',
    'Chloe': '<same_as_state />',
    'Jack': '<same_as_state />',
    'Lily': '<same_as_state />'},
   'actions': {'Carter': 'Carter moved the tomato to the red_drawer',
    'Isabella': 'none',
    'Chloe': 'none',
    'Jack': 'none',
    'Lily': 'none'}},
